<a href="https://colab.research.google.com/github/farhadhsn8/implementation_ML_algorithms/blob/master/bagging_randomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

In [4]:
class node:
  def __init__(self,features , targets , purityTreshold):
    self.purityTreshold = purityTreshold
    self.left = None
    self.right = None
    self.features = features
    self.targets = targets
    self.accuricy = None
    self.label = None
    self.isTerminalNode = False

  def computeAccuricy(self):
    self.accuricy = np.bincount(self.targets).max() / self.targets.shape[0]

  def generateLabel(self):
    if (self.accuricy >= self.purityTreshold ):
      self.label = np.bincount(self.targets).argmax()
      self.isTerminalNode = True

  def findBestFeature(self):
    max_purity = [0,0] 
    for i in range(self.features.shape[1]):
      feature_trshold = self.findBestTresholdPlaceOnFeature(i)
      #======================================!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    

  def findBestTresholdPlaceOnFeature(self, feacherNumber):
    FT = np.hstack((self.features[feacherNumber],self.targets))
    max_acc = [0 , 0]
    for i in range(FT.shape[0]):
      smaller_filter = FT[:,0] <= FT[i,0]
      bigger_filter = FT[:,0] > FT[i,0]
      smaller_arr = FT[smaller_filter]
      bigger_arr = FT[bigger_filter] 
      correct = np.bincount(smaller_arr)[np.bincount(smaller_arr).argmax()] + np.bincount(bigger_arr)[np.bincount(bigger_arr).argmax()]
      acc = correct / FT.shape[0] 
      if acc > max_acc[1]:
        max_acc = [i,acc]
    return max_acc[0]



    


class decisionTree:
  def __init__(self,features , targets , purityTreshold):
    self.head = node(features , targets , purityTreshold)



# create root
# root = Node(1)

 
# root.left      = Node(2);
# root.right     = Node(3);
   
# root.left.left  = Node(4);


In [7]:
x = np.array([1,2,3,4,5,1,2,1,1,1])
print("Original array:")
print(x)
  
print("Most frequent value in the above array:")
print(np.bincount(x)[np.bincount(x).argmax()])

Original array:
[1 2 3 4 5 1 2 1 1 1]
Most frequent value in the above array:
5
